In [ ]:
import pandas as pd
import altair as alt
from vega_datasets import data

In [ ]:
cities_df = pd.read_csv("geodata.csv")
pickle_df = pd.read_pickle("df.pickle")

cities_df.head()

In [ ]:
joined_df = pickle_df.merge(cities_df, left_on='Zillow', right_on='city')
joined_df.head()

In [ ]:
joined_df.filename.unique()

In [ ]:
states = alt.topo_feature(data.us_10m.url, feature='states')

background = alt.Chart(states).mark_geoshape(
    fill='#f0f0f0',
    stroke='#ddd'
).project('albersUsa').properties(
    width=500,
    height=300
)

mid_tier_all = 'Metro_zhvi_uc_sfrcondo_tier_0.33_0.67_month'
start_df = joined_df[(joined_df.Year==2010) & (joined_df.filename == mid_tier_all) & (joined_df.Month == 1)]
points = alt.Chart(start_df).mark_circle(size=20, opacity=0.8).encode(
    longitude='lng',
    latitude='lat',
    tooltip=['city', 'Price'],
    color=alt.Color(
        'Price', scale=alt.Scale(type='log', scheme='redblue'), legend=alt.Legend(title=None, format='$,'),
        sort='descending'
    ),
)

(background + points).properties(title={
    'text': 'Mid-Tier Home Price, 2010',
    'fontSize': 20,
    'anchor': 'start',
})

In [ ]:
end_df = joined_df[(joined_df.Year==2021) & (joined_df.filename == mid_tier_all) & (joined_df.Month == 1)]
points = alt.Chart(end_df).mark_circle(size=20, opacity=0.8).encode(
    longitude='lng',
    latitude='lat',
    tooltip=['city', 'Price'],
    color=alt.Color(
        'Price', scale=alt.Scale(type='log', scheme='redblue'), legend=alt.Legend(title=None, format="$,"),
        sort='descending'
    ),
)

(background + points).properties(title={
    'text': 'Mid-Tier Home Price, 2021',
    'fontSize': 20,
    'anchor': 'start',
})

In [ ]:
cols = ['Zillow', 'Price']
change_df = pd.merge(
    start_df[['lat', 'lng'] + cols], end_df[cols], on='Zillow', suffixes=['_2010', '_2021'], how='inner'
)
change_df['Change'] = (change_df['Price_2021'] / change_df['Price_2010']) - 1
change_df.head()

In [ ]:
points = alt.Chart(change_df).mark_circle(size=20, opacity=0.8).encode(
    longitude='lng',
    latitude='lat',
    tooltip=['Zillow', 'Change'],
    color=alt.Color(
        'Change', scale=alt.Scale(scheme='redblue'), legend=alt.Legend(format="%", title=None), sort='descending'
    ),
)

(background + points).properties(title={
    'text': 'Price Change 2010-2021',
    'fontSize': 20,
    'anchor': 'start',
})